# Advanced features

Here are yet more tools that the average user won't need, but might come in handy one day.

<div class="alert alert-info">
    
Click [here](https://mybinder.org/v2/gh/sciris/sciris/HEAD?labpath=docs%2Ftutorials%2Ftut_advanced.ipynb) to open an interactive version of this notebook.
    
</div>


## Nested dictionaries

TBC

## Context blocks

Sciris contains two context block (i.e. "`with ... as`") functions. `sc.capture()` captures all text output to a variable:

In [ ]:
import sciris as sc
import numpy as np

def verbose_func(n=200):
    for i in range(n):
        print(f'Here are 5 random numbers: {np.random.rand(5)}')

with sc.capture() as output:
    verbose_func()

lines = output.splitlines()
target = '777'
for l,line in enumerate(lines):
    if target in line:
        print(f'Found target {target} on line {l}: {line}')

The other function, `sc.tryexcept()`, is a more compact way of writing `try ... except` blocks, and gives detailed control of error handling:

In [ ]:
def fickle_func(n=1):
    for i in range(n):
        rnd = np.random.rand()
        if rnd < 0.005:
            raise ValueError(f'Value {rnd} too small')
        elif rnd > 0.99:
            raise RuntimeError(f'Value {rnd} too big')

sc.heading('Simple usage, exit gracefully at first exception')
with sc.tryexcept():
    fickle_func(n=1000)

sc.heading('Store all history')
tryexc = None
for i in range(1000):
    with sc.tryexcept(history=tryexc, verbose=False) as tryexc:
        fickle_func()
tryexc.to_df()

## Interpolation and optimization

TBC

## Animation

TBC